In [1]:
import os
import pickle
import numpy as np
import random
import pandas as pd
import torch
import torch.nn as nn

In [2]:
from IPython.display import clear_output
from factory.AutoVC_Adjust import AutoVC_Adjust

In [3]:
from factory.MetaDV import MetaDV

In [4]:
ROOT = "train_data_en_vctk_ver2"
device = "cuda:0"

In [5]:
erroment_uttr_idx = 16
max_uttr_idx = 60
num_speaker = 40

In [6]:
all_speaker = sorted(next(iter(os.walk(f'{ROOT}')))[1][:num_speaker])
all_speaker

['p225',
 'p226',
 'p227',
 'p228',
 'p229',
 'p230',
 'p231',
 'p232',
 'p233',
 'p234',
 'p236',
 'p237',
 'p238',
 'p239',
 'p240',
 'p241',
 'p243',
 'p244',
 'p245',
 'p246',
 'p247',
 'p248',
 'p249',
 'p250',
 'p251',
 'p252',
 'p253',
 'p254',
 'p255',
 'p256',
 'p257',
 'p258',
 'p259',
 'p260',
 'p261',
 'p262',
 'p263',
 'p264',
 'p265',
 'p266']

In [7]:
# 評估用
C = torch.load('model/static/metadv_en_openslr.pt').to(device)

In [8]:
# 要用原本訓練的來生聲音
metadata = pickle.load(open(f'{ROOT}/train.pkl', "rb"))[:num_speaker]

In [9]:
metadata_copy = []
for data in metadata:
    if data[0] in all_speaker:
        metadata_copy.append(data)
metadata = sorted(metadata_copy)
for data in metadata:
    print(data[0])

p225
p226
p227
p228
p229
p230
p231
p232
p233
p234
p236
p237
p238
p239
p240
p241
p243
p244
p245
p246
p247
p248
p249
p250
p251
p252
p253
p254
p255
p256
p257
p258
p259
p260
p261
p262
p263
p264
p265
p266


In [10]:
org_ad = AutoVC_Adjust(44,256,512,22).to(device)
org_ad.load_state_dict(torch.load("model/autovc_adjust_en.pt", map_location=device))

<All keys matched successfully>

In [11]:
def get_trans_mel(model,sample,emb_org,emb_trg):
    return model(sample.to(device),emb_org.to(device),emb_trg.to(device))[1][0,:,:].detach().cpu()

In [12]:
def get_crop_mel(tmp,len_crop=176):
    if tmp.shape[0] < len_crop:
        pad_size = int(len_crop - tmp.shape[0])
        npad = [(0, 0)] * tmp.ndim
        npad[0] = (0, pad_size)
        tmp = np.pad(tmp, pad_width=npad, mode='constant', constant_values=0)
        melsp = torch.from_numpy(tmp)
            
    elif tmp.shape[0] == len_crop:
        melsp = torch.from_numpy(tmp)
    else:
        left = np.random.randint(0, tmp.shape[0] - len_crop)
        melsp = torch.from_numpy(tmp[left : left + len_crop, :])
    return melsp.to(device)

In [13]:
import torch.nn.functional as F
def get_dv(speaker):
    all_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][:erroment_uttr_idx]
    _dv = torch.zeros((1,256))
    for uttr in all_uttr:
        mel = torch.from_numpy(np.load(f'{ROOT}/{speaker}/{uttr}')).unsqueeze(0).to(device)
        if mel.shape[1] < 176:
            pad_size = int(176 - mel.shape[1])
            mel = F.pad(mel,(0,0,int(pad_size/2),int(pad_size/2)),'constant',0)
            print(mel.shape)
 
        _dv += C(mel)[1].detach().cpu()
    _dv = _dv/(erroment_uttr_idx+1)
    random_uttr = next(iter(os.walk(f'{ROOT}/{speaker}')))[2][random.randint(erroment_uttr_idx,max_uttr_idx)]
    mel = get_crop_mel(np.load(f'{ROOT}/{speaker}/{random_uttr}')).unsqueeze(0)
    emb_org = _dv.to(device) 
    return mel,emb_org,speaker

In [14]:
all_dv = []
for i,speaker in enumerate(all_speaker):
    all_dv.append(get_dv(speaker))

torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])


In [15]:
def get_cos_rc(cos_res):
    rc = 0.0
    rc_ = 0.0
    N = len(cos_res)
    for i in range(N):
        total = sum(cos_res[i][i])
        others = (total - cos_res[i][i][i])/(N-1)
        rc_ += others
        rc += cos_res[i][i][i]
    return rc/N,rc_/N

In [16]:
def get_cos_trans(cos_res):
    trans = []
    trans_ = []
    N = len(cos_res)
    for i in range(N):
        trans_cos = np.sum(np.diagonal(cos_res[i])) - cos_res[i][i][i]
        trans.append(trans_cos / (N-1))
        trans_.append(  (np.sum(cos_res[i]) - np.sum(np.diagonal(cos_res[i]))) / ((N-1)*N))
    
    return sum(trans)/len(trans),sum(trans_)/len(trans_)

In [19]:
cos_reslut_rc_ad = np.zeros((num_speaker,num_speaker,num_speaker)) 
for i,data in enumerate(metadata):
    sp_s = data[0]
    source_mel = get_crop_mel(np.load(f"{ROOT}//{data[random.randint(erroment_uttr_idx,max_uttr_idx)]}")).unsqueeze(0)
    emb_org = torch.from_numpy(data[1]).unsqueeze(0)
    print(f"Now Processing --- {sp_s}")
    for j,data in enumerate(metadata):
        sp_o = data[0]
        emb_trg = torch.from_numpy(data[1]).unsqueeze(0)
        trans_mel_ad = get_trans_mel(org_ad,source_mel,emb_org,emb_trg).to(device)
        ad_dv = C(trans_mel_ad)[1]
        
        if i==j:
            print(f"Reconstruct ---- {sp_s} to {sp_o}")
        else:
            print(f"Trans --- {sp_s} to {sp_o}")
        for k,data in enumerate(all_dv):
            _,emb,_ = data 
            cos = torch.clamp(nn.functional.cosine_similarity(ad_dv,emb, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
            cos_reslut_rc_ad[i][j][k] = cos

Now Processing --- p225
Reconstruct ---- p225 to p225
Trans --- p225 to p226
Trans --- p225 to p227
Trans --- p225 to p228
Trans --- p225 to p229
Trans --- p225 to p230
Trans --- p225 to p231
Trans --- p225 to p232
Trans --- p225 to p233
Trans --- p225 to p234
Trans --- p225 to p236
Trans --- p225 to p237
Trans --- p225 to p238
Trans --- p225 to p239
Trans --- p225 to p240
Trans --- p225 to p241
Trans --- p225 to p243
Trans --- p225 to p244
Trans --- p225 to p245
Trans --- p225 to p246
Trans --- p225 to p247
Trans --- p225 to p248
Trans --- p225 to p249
Trans --- p225 to p250
Trans --- p225 to p251
Trans --- p225 to p252
Trans --- p225 to p253
Trans --- p225 to p254
Trans --- p225 to p255
Trans --- p225 to p256
Trans --- p225 to p257
Trans --- p225 to p258
Trans --- p225 to p259
Trans --- p225 to p260
Trans --- p225 to p261
Trans --- p225 to p262
Trans --- p225 to p263
Trans --- p225 to p264
Trans --- p225 to p265
Trans --- p225 to p266
Now Processing --- p226
Trans --- p226 to p225
Re

## Real Data

In [20]:
cos_gt = np.zeros((num_speaker,num_speaker))
for i,speaker in enumerate(all_speaker):
    mel,_,_ = get_dv(speaker)
    _dv = C(mel)[1].detach().cpu()
    for j,data in enumerate(all_dv):
        dv = data[1].detach().cpu()
        cos = torch.clamp(nn.functional.cosine_similarity(_dv,dv, dim=1, eps=1e-8),min=0.0).detach().cpu().numpy()[0].astype(np.float32)
        cos_gt[i][j] = cos
np.diagonal(cos_gt).mean(), (np.sum(cos_gt) - (np.sum(np.diagonal(cos_gt)))) /(num_speaker*(num_speaker-1))

torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 175, 80])
torch.Size([1, 176, 80])
torch.Size([1, 176, 80])
torch.Size([1, 175, 80])


(0.9255942463874817, 0.7290968976914882)

## Reconstruct

In [21]:
get_cos_rc(cos_reslut_rc_ad),get_cos_trans(cos_reslut_rc_ad)

((0.9382708445191383, 0.7484937824690953),
 (0.9070077665341207, 0.7688545159876155))

## Trans

In [1]:
rc_reslut_rc_ad = []
_rc_reslut_rc_ad = []
trans_reslut_rc_ads = []
_trans_reslut_rc_ads = []

## Cos thresholds

In [25]:
thresholds = np.arange(0,1.01,0.01).astype(np.float32)

In [26]:
threshold = 0.8

In [27]:
def get_recall(cos_result,threshold=0.8):
    TP = 0.0
    FN = 0.0
    for cos_res in cos_result:
        trans_ = np.diagonal(cos_res)
        for trans in trans_:
            if trans > threshold:
                TP +=1
            else:
                FN += 1
    return (TP/(TP+FN))

In [28]:
get_recall(cos_reslut_rc_ad,threshold)

1.0

In [29]:
get_recall(cos_reslut_rc_org,threshold)

0.69

In [81]:
#get_recall(cos_reslut_rc_dc,threshold),

(0.80625,)

In [83]:
#get_recall(cos_reslut_rc_conv,threshold),

(0.96,)